## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-05-46-37 +0000,bbc,Musk's AI firm says it's removing 'inappropria...,https://www.bbc.com/news/articles/c4g8r34nxeno
1,2025-07-09-05-42-52 +0000,nyt,"Norman Tebbit, Consertavive Force in Britain a...",https://www.nytimes.com/2025/07/08/world/europ...
2,2025-07-09-05-31-35 +0000,nyt,Wildfire on Edge of Marseille Injures More Tha...,https://www.nytimes.com/2025/07/09/world/europ...
3,2025-07-09-05-15-52 +0000,bbc,Families anxious over safety of Indians kidnap...,https://www.bbc.com/news/articles/c9vrj0jn8v8o
4,2025-07-09-05-04-56 +0000,nypost,Trump insists ‘no amnesty’ for migrant farmwor...,https://nypost.com/2025/07/09/us-news/trump-in...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2299/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,trump,25
75,tariffs,8
122,new,6
133,texas,5
577,scandal,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
46,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,57
10,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...,54
90,2025-07-08-13-59-19 +0000,nyt,Trump’s New Trade Threats Set Off Global Scram...,https://www.nytimes.com/2025/07/08/business/ec...,48
92,2025-07-08-13-44-50 +0000,nyt,Trump Targets Japan With 25% Tariffs After Sev...,https://www.nytimes.com/2025/07/08/business/tr...,46
50,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...,45


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
46,57,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
20,28,2025-07-09-01-00-00 +0000,wsj,Two Republicans named Kevin are vying to be th...,https://www.wsj.com/economy/central-banking/tr...
18,23,2025-07-09-01-49-09 +0000,nypost,College group Zohran Mamdani co-founded welcom...,https://nypost.com/2025/07/08/us-news/college-...
30,22,2025-07-08-23-26-50 +0000,nypost,Mayor Eric Adams builds out re-election campai...,https://nypost.com/2025/07/08/us-news/mayor-er...
14,22,2025-07-09-02-52-23 +0000,nypost,Rep. Ilhan Omar’s activist daughter unemployed...,https://nypost.com/2025/07/08/us-news/rep-ilha...
31,22,2025-07-08-23-16-20 +0000,nypost,Sen. Ted Cruz pictured at Parthenon on Greek v...,https://nypost.com/2025/07/08/us-news/ted-cruz...
10,21,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...
32,19,2025-07-08-23-16-14 +0000,nypost,"Ex-‘Squad’ congresswoman Cori Bush, Missouri D...",https://nypost.com/2025/07/08/us-news/dem-rep-...
59,19,2025-07-08-19-28-44 +0000,nypost,Zelensky’s right-hand man thanks Trump for rev...,https://nypost.com/2025/07/08/us-news/zelensky...
50,17,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
